In [3]:
# To default to float division and print function.
from __future__ import (division, print_function)

# Core python libraries
import csv
import re
import math

# External libraries.
import nltk
import numpy
import pandas
import scipy
import sklearn

# Tensorflow and related.
import tensorflow

# For fitting streamlined tensor flow models.
import tflearn

# So you know when this code block finishes.
print ("Done")

Done


In [7]:
data = pandas.read_csv('data/train_medium.csv')

X_train_index, X_test_index, Y_train, Y_test = sklearn.cross_validation.train_test_split(
    data.index, data['LogSalaryNormalized'], test_size=.33, random_state=42)

# Keep train and test as pandas dataframes.
X_train = data.iloc[X_train_index]
X_test = data.iloc[X_test_index]

# Uncomment this next line if you want to check the data.
# data.head()

print ("Done")

Done


In [12]:
MIN_WORD_FREQUENCY = 15

count_vect = sklearn.feature_extraction.text.CountVectorizer(
  stop_words='english', min_df=MIN_WORD_FREQUENCY)

# Here I am count vectoring the full description field. But in theory
# any text field can be processed this way.
X_train_full_description_counts = count_vect.fit_transform(X_train['FullDescription'])
X_test_full_description_counts = count_vect.transform(X_test['FullDescription'])

print(X_train_full_description_counts.shape)
print(X_test_full_description_counts.shape)
#TODO (max): supress the VisibleDeprecationWarning here from numpy. It is being thrown by the package.

(670, 786)
(330, 786)


In [18]:
def original_salary_scale(log_value):
    return math.exp(log_value)

def mean_absolute_error_salary_scale(y_test, y_predicted):
    return sklearn.metrics.mean_absolute_error(
        numpy.exp(y_test), numpy.exp(y_predicted))

# Guess the average. Create an empty vector of the desired shape.
average_guess = numpy.empty(Y_test.shape)
average_guess.fill(numpy.mean(Y_train))

mean_guess = average_guess[0]
print('Mean salary value in training set is  £{:10.2f}'.format(original_salary_scale(mean_guess)))
average_guess_mae = mean_absolute_error_salary_scale(Y_test, average_guess)
print('Guess the average Mean Absolute Error: {:10.4f}'.format(average_guess_mae))

# SGD Needs normalized inputs
# TODO: Suppress DataConversionWarning from converting ints to floats here.
normalizer = sklearn.preprocessing.Normalizer(norm='l1')
X_train_norm = normalizer.fit_transform(X_train_full_description_counts.astype('float64'))
X_test_norm = normalizer.transform(X_test_full_description_counts.astype('float64'))

# We want a stochastic gradient descent with l1 norm.
sgd = sklearn.linear_model.SGDRegressor(alpha=.005, penalty='l1', n_iter=10000)
sgd.fit(X_train_norm, Y_train)
sgd_predictions = sgd.predict(X_test_norm)
sgd_mae = mean_absolute_error_salary_scale(Y_test, sgd_predictions)
print('SGDRegressor Mean Absolute Error: {:10.4f}'.format(sgd_mae))
#TODO (any): wonder why this is so inaccurate/wrong

Mean salary value in training set is  £  25941.22
Guess the average Mean Absolute Error:  9396.2364
SGDRegressor Mean Absolute Error:  9320.6531


In [17]:
print(sgd.intercept_)
print (mean_guess)

[ 9.80475551]
10.1635886033


In [19]:
# Please see http://tflearn.org/examples/
#dnn_model = tflearn.models.dnn.DNN([3,3,3,1], 0
X = X_train_full_description_counts.todense().astype(float)
m, n = numpy.shape(X)
Y = Y_train
Y = Y.reshape(m, 1).astype(float)
#print (Y.shape)


#input_layer = tflearn.input_data(shape=[None, n], name='input')
#linear = tflearn.fully_connected(input_layer, 16)

#dnn_regressor = tensorflow.contrib.learn.DNNRegressor(hidden_units=[128, 1])
#dnn_regressor.fit(feed_dict={X: X, Y: Y}, batch_size=100, steps=1000)

#regression = tflearn.regression(linear, optimizer='sgd', loss='mean_square',
#                                metric='R2', learning_rate=0.01)
#m = tflearn.DNN(regression)
#m.fit(X, Y, n_epoch=1000, show_metric=True, snapshot_epoch=False)


In [20]:
"""
W = tensorflow.Variable(tensorflow.zeros([m,1]))
b = tensorflow.Variable(tensorflow.zeros([1]))

b = tensorflow.Print(b, [b], "Bias: ")
W = tensorflow.Print(W, [W], "Weights: ")

y = tensorflow.add(tensorflow.matmul(X, W), b)
y = tensorflow.Print(y, [y], "y: ")

loss = tensorflow.reduce_sum(tensorflow.square(y - y_data)) / (2 * m)
loss = tensorflow.Print(loss, [loss], "loss: ")
optimizer = tensorflow.train.GradientDescentOptimizer(.001)

train = optimizer.minimize(loss)

init = tensorflow.initialize_all_variables()

sess = tensorflow.Session()
sess.run(init)                                

for i in range(10):
  sess.run(train)
  #if i % 20 == 0:
        #print(sess.run(W), sess.run(b))  

sess.close()
"""

'\nW = tensorflow.Variable(tensorflow.zeros([m,1]))\nb = tensorflow.Variable(tensorflow.zeros([1]))\n\nb = tensorflow.Print(b, [b], "Bias: ")\nW = tensorflow.Print(W, [W], "Weights: ")\n\ny = tensorflow.add(tensorflow.matmul(X, W), b)\ny = tensorflow.Print(y, [y], "y: ")\n\nloss = tensorflow.reduce_sum(tensorflow.square(y - y_data)) / (2 * m)\nloss = tensorflow.Print(loss, [loss], "loss: ")\noptimizer = tensorflow.train.GradientDescentOptimizer(.001)\n\ntrain = optimizer.minimize(loss)\n\ninit = tensorflow.initialize_all_variables()\n\nsess = tensorflow.Session()\nsess.run(init)                                \n\nfor i in range(10):\n  sess.run(train)\n  #if i % 20 == 0:\n        #print(sess.run(W), sess.run(b))  \n\nsess.close()\n'